In [1]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

In [2]:
import openai
openai.organization = organization
openai.api_key = api_key

In [3]:
gpt_model_name = "gpt-3.5-turbo"

In [4]:
def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message

In [5]:
basic_completion("Write a haiku about being a dungeon master")

<OpenAIObject at 0x7f22143bfab0> JSON: {
  "content": "Rolling the dice high\nCrafting each story anew\nMaster of the depths",
  "role": "assistant"
}

In [58]:
system_prompt = '''
You are a dungeon master assistant. You will create and improvise world content, behaviors, and events.

World lore and game mechanics should be compatible with dungeons and dragons standard fifth edition (5e).

All response MUST be in valid YAML format, and MUST NOT contain any other characters.
The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
'''

import yaml

def system_completion(user_prompt):
    amended_user_prompt = f"""
    {user_prompt}

    The response MUST be in valid YAML format, and MUST NOT contain any other characters.
    The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
    """

    call = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': amended_user_prompt}
        ],
    )
    raw_message = call.choices[0].message['content']
    parsed_message = yaml.safe_load(raw_message)
    return (raw_message, parsed_message)


In [53]:
(raw_name, parsed_name) = system_completion("Generate a name for a fantasy world")
print(raw_name)

name: Alderathia


In [66]:
def range(pair): return f"at least {pair[0]} and no greater than {pair[1]}"

def geography_prompt(continents=(3,3), regions=(4,7)): return f'''
Generate the geography of a fictional fantasy world, that will be the setting for the game.

There MUST be {range(continents)} continents in the world.
There MUST be {range(regions)} regions per continent.

The response must be in the following structure. 

continents:
    location_in_world: <string>
    regions:
        location_in_continent: <string>
        topography: <string>
        climate: <string>
        biome: <string>
        size: <string>
        name: <string>
    name: <string>
'''

geography_prompt()

'\nGenerate the geography of a fictional fantasy world, that will be the setting for the game.\n\nThere MUST be at least 3 and no greater than 3 continents in the world.\nThere MUST be at least 4 and no greater than 7 regions per continent.\n\nThe response must be in the following structure. \n\ncontinents:\n    location_in_world: <string>\n    regions:\n        location_in_continent: <string>\n        topography: <string>\n        climate: <string>\n        biome: <string>\n        size: <string>\n        name: <string>\n    name: <string>\n'

In [68]:
(raw_geography, parsed_geography) = system_completion(geography_prompt())
print(raw_geography)

continents:
  - 
    location_in_world: East Pole
    name: Almara
    regions:
      - 
        location_in_continent: North
        topography: Mountains
        climate: Arctic
        biome: Tundra
        size: Small
        name: Frosthold
      - 
        location_in_continent: East
        topography: Forests
        climate: Temperate
        biome: Taiga
        size: Medium
        name: Linvendale
      - 
        location_in_continent: South
        topography: Plains
        climate: Dry
        biome: Grassland
        size: Large
        name: Aridum
      - 
        location_in_continent: West
        topography: Swamps
        climate: Tropical
        biome: Wetland
        size: Medium
        name: Murkwood
  - 
    location_in_world: Equator
    name: Isalor
    regions:
      - 
        location_in_continent: North
        topography: Mountains
        climate: Arctic
        biome: Tundra
        size: Medium
        name: Frostfall
      - 
        location_in_

In [71]:
parsed_geography

{'continents': [{'location_in_world': 'East Pole',
   'name': 'Almara',
   'regions': [{'location_in_continent': 'North',
     'topography': 'Mountains',
     'climate': 'Arctic',
     'biome': 'Tundra',
     'size': 'Small',
     'name': 'Frosthold'},
    {'location_in_continent': 'East',
     'topography': 'Forests',
     'climate': 'Temperate',
     'biome': 'Taiga',
     'size': 'Medium',
     'name': 'Linvendale'},
    {'location_in_continent': 'South',
     'topography': 'Plains',
     'climate': 'Dry',
     'biome': 'Grassland',
     'size': 'Large',
     'name': 'Aridum'},
    {'location_in_continent': 'West',
     'topography': 'Swamps',
     'climate': 'Tropical',
     'biome': 'Wetland',
     'size': 'Medium',
     'name': 'Murkwood'}]},
  {'location_in_world': 'Equator',
   'name': 'Isalor',
   'regions': [{'location_in_continent': 'North',
     'topography': 'Mountains',
     'climate': 'Arctic',
     'biome': 'Tundra',
     'size': 'Medium',
     'name': 'Frostfall'},
   

In [73]:
geography_continent_context = []

for continent in parsed_geography['continents']:
    name = continent['name']
    location_in_world = continent['location_in_world']
    def get_biome(region): return region['biome']
    biomes = map(get_biome, continent['regions'])
    biomes = ', '.join(biomes)
    line = f"The continent {name} located in the {location_in_world} with {biomes} biomes."
    geography_continent_context.append(line)

geography_continent_context = '\n'.join(geography_continent_context)
print(geography_continent_context)

The continent Almara located in the East Pole with Tundra, Taiga, Grassland, Wetland biomes.
The continent Isalor located in the Equator with Tundra, Taiga, Rainforest, Tropical Forest, Desert, Deciduous Forest biomes.
The continent Balthor located in the West Pole with Tundra, Taiga, Desert, Rainforest biomes.


In [77]:
def civilizations_prompt(geography_context, civilizations=(4,5), settlements=(3,6)): return f"""
Generate the major fictional fantasy civilizations in the world and their major cities.

There MUST be {range(civilizations)} civilizations in the world.
There MUST be {range(settlements)} major cities per civilization.

The civlizations and settlements should exist in the world's geopgrahy, defined by the below section:

```
{geography_context}
```

The response must be in the following structure.

civilizations:
    location_continent: <reference to continent in world>
    cities:
        location_region: <reference to region in world>
        economy_description: <string>
        name: <string>
    ruling_faction: <string>
    name: <string>
"""

civilizations_prompt(raw_geography)

"\nGenerate the major fictional fantasy civilizations in the world and their major cities.\n\nThere MUST be at least 4 and no greater than 5 civilizations in the world.\nThere MUST be at least 3 and no greater than 6 major cities per civilization.\n\nThe civlizations and settlements should exist in the world's geopgrahy, defined by the below section:\n\n```\ncontinents:\n  - \n    location_in_world: East Pole\n    name: Almara\n    regions:\n      - \n        location_in_continent: North\n        topography: Mountains\n        climate: Arctic\n        biome: Tundra\n        size: Small\n        name: Frosthold\n      - \n        location_in_continent: East\n        topography: Forests\n        climate: Temperate\n        biome: Taiga\n        size: Medium\n        name: Linvendale\n      - \n        location_in_continent: South\n        topography: Plains\n        climate: Dry\n        biome: Grassland\n        size: Large\n        name: Aridum\n      - \n        location_in_continent:

In [78]:
(raw_civilizations, parsed_civlizations) = system_completion(civilizations_prompt(raw_geography))
print(raw_civilizations)

civilizations:
  - 
    location_continent: Almara
    cities:
      - 
        location_region: Frosthold
        economy_description: Known for their skilled blacksmiths and hunters, Frosthold's economy relies heavily on the trade of furs, meat, and weapons.
        name: Iceforge
      - 
        location_region: Linvendale
        economy_description: Linvendale serves as a hub for trade and scholarship, their economy built around agriculture, lumber, and knowledge industries.
        name: Starhaven
      -
        location_region: Aridum
        economy_description: Built on the natural resources of the surrounding grasslands, Aridum has become a center for trade in textiles, spices, and luxury goods.
        name: Oasis City
      - 
        location_region: Murkwood
        economy_description: Despite the dangers of the surrounding swamp, Murkwood serves as one of the main sources of fishing, lumber, and exotic plants and mushrooms.
        name: Siltmarsh
    ruling_faction: 